In [1]:
import folium
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

In [2]:
df = pd.read_csv("Active Hotel Residentials at the End of 2019.xlsx - Report_BL_ByNAICCode_CLLE(8).csv")

In [3]:
(df.Longitude.min() + df.Longitude.max()) / 2, (df.Latitude.min() + df.Latitude.max()) / 2

(-78.48748115000001, 38.037425119999995)

In [4]:
df = df[(df.Latitude.notnull()) & (df.Longitude.notnull())]
df

,Trading As Name,Address as given,Zip,Address,Latitude,Longitude,Zoning,2020 Zoning Permit,2019 Business License
0,Claasen Johanna A,1010 LOCUST LN 22901,22901,1010 LOCUST LN,38.044300,-78.456867,R-1S,Yes,Yes
1,Daggett Todd & Dewaele Audrey,1012 ST CHARLES AVE 22901,22901,1012 ST CHARLES AVE,38.041863,-78.463529,R-1,Yes,Yes
2,"DUDA, ERIK",1014 ST CHARLES AVE 22901,22901,1014 ST CHARLES AVE,38.041978,-78.463391,R-1,Yes,Yes
3,Brady Ryan,102 Somesso Ct 22902,22902,102 Somesso Ct,38.023299,-78.488853,PUD,Yes,Yes
4,Bhandari Chhabi,1028 ST CLAIR AVE 22901,22901,1028 ST CLAIR AVE,38.040771,-78.459774,R-1S,Yes,Yes
5,"GERBER, MATTHEW J",104 BUCKINGHAM RD 22903,22903,104 BUCKINGHAM RD,38.048362,-78.494731,R-1U,Yes,Yes
6,"JOYCE, MALLORY B",104 OAK LAWN CT 22903,22903,104 OAK LAWN CT,38.015816,-78.510377,R-1S,Yes,Yes
7,Blue Heron AirBnb,106 Burnet Way 22902,22902,106 Burnet Way,38.023730,-78.488187,PUDH,Yes,Yes
8,Fry House LLC,109 RAYMOND AVE 22903,22903,109 RAYMOND AVE,38.023612,-78.513979,R-1S,Yes,Yes
9,Booren Leslie M,110 THOMAS DR 22903,22903,110 THOMAS DR,38.026166,-78.506240,R-2,Yes,Yes


In [5]:
df.groupby(["2020 Zoning Permit", "2019 Business License"]).count()

Trading As Name  Address as given  \
2020 Zoning Permit 2019 Business License                                      
No                 Yes                                174               174   
Yes                No                                  13                13   
                   Yes                                 81                81   

                                          Zip  Address  Latitude  Longitude  \
2020 Zoning Permit 2019 Business License                                      
No                 Yes                    174      173       174        174   
Yes                No                      13       13        13         13   
                   Yes                     81       81        81         81   

                                          Zoning  
2020 Zoning Permit 2019 Business License          
No                 Yes                       174  
Yes                No                         13  
                   Yes                        81

In [6]:
# geometry = [Point(xy) for xy in zip(df.Longitude, df.Latitude)]
# df = df.drop(['Longitude', 'Latitude'], axis=1)
# crs = {'init': 'epsg:4326'}
# gdf = gpd.GeoDataFrame(df, crs=crs, geometry=geometry)
# gdf

In [7]:
folium_map = folium.Map(location=[38.037425119999995, -78.48748115000001],
                        zoom_start=14,
                        tiles="CartoDB positron")
    
for idx, zone in enumerate(list(df.Zoning.unique())):
    fg=folium.FeatureGroup(name="Zone: {}".format(zone))
    tmp = df[df.Zoning == zone]
    
    for idx, row in tmp.iterrows():
        popup_text = "Name: {}<br> Address: {}<br> Zoning: {}"
        popup_text = popup_text.format(row["Trading As Name"],row["Address"],row["Zoning"])
        
        if row["2019 Business License"] == "No" and row["2020 Zoning Permit"] == "Yes":
            fg.add_child(folium.CircleMarker(location=(row["Latitude"], row["Longitude"]),
                                             popup=popup_text,
                                             fill=True,
                                             color="red"))
        elif row["2019 Business License"] == "Yes" and row["2020 Zoning Permit"] == "No":
            fg.add_child(folium.CircleMarker(location=(row["Latitude"], row["Longitude"]),
                                             popup=popup_text,
                                             fill=True,
                                             color="orange"))
        elif row["2019 Business License"] == "Yes" and row["2020 Zoning Permit"] == "Yes":
            fg.add_child(folium.CircleMarker(location=(row["Latitude"], row["Longitude"]),
                                             popup=popup_text,
                                             fill=True,
                                             color="blue"))


        
    folium_map.add_child(fg)
    
lc = folium.LayerControl().add_to(folium_map)

folium_map

In [8]:
folium_map.save("my_map.html")